# Generación de Texto usando una  Recurrent Neuronal Network del tipo RNN básico, LSTM o GRU
Basado en https://www.tensorflow.org/tutorials/text/text_generation

1) Cargar las librerías:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

from google.colab import files 
import io

print("Librerías cargadas")

Librerías cargadas


2) Cargar el texto base a procesar:

*   desde Google Drive:

*Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.*

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demoRNN'

In [0]:
# levanta el archivo de texto del Drive para procesar
text = open("".join([path, "/RNN Cantata del adelantado Don Rodrigo Diaz de Carreras - preparado.txt"]), 'rb').read().decode(encoding='utf-8')

print("Archivo cargado")



*   Subiendolo desde el disco local:


In [2]:
# Seleccionar y subir archivo de texto del disco local al Drive para procesar
uploaded = files.upload()


Saving RNN proverbios.txt to RNN proverbios.txt


In [3]:
# carga el texto a procesar y lo codifica en formato py2
text = str(str(uploaded[list(uploaded.keys())[0]]))

print("Archivo cargado")

Archivo cargado


Una vez cargado el archivo muestra sus estadísticas:

In [4]:
# length of text is the number of characters in it
print ('Tamaño total del texto: {} caracteres'.format(len(text)))

# muestra los primeros 250 caracteres del texto
print("Ejemplo: \n", text[:250])

Tamaño total del texto: 59988 caracteres
Ejemplo: 
 b'\xef\xbb\xbfa caballo regalado no se le mira el colmillo\r\na caballo regalado no se le miran los dientes\r\na cada chancho le llega su san marti\r\na confesion de parte relevo de prueba\r\nalli donde fueres haz lo que vieres\r\ndondequiera que fue


3) Preparar el texto base a procesar:

In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} caracteres distintos detectados'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# Muestra ejemplo de cómo se mapean los caracteres a valores numéricos
print ('{} <-------- > {}'.format(repr(text[:13]), text_as_int[:13]))

40 caracteres distintos detectados
{
  ' ' :   0,
  '"' :   1,
  "'" :   2,
  '/' :   3,
  '0' :   4,
  '1' :   5,
  '2' :   6,
  '3' :   7,
  '8' :   8,
  '9' :   9,
  '?' :  10,
  '[' :  11,
  '\\':  12,
  ']' :  13,
  'a' :  14,
  'b' :  15,
  'c' :  16,
  'd' :  17,
  'e' :  18,
  'f' :  19,
  ...
}
"b'\\xef\\xbb\\xb" <-------- > [15  2 12 37 18 19 12 37 15 15 12 37 15]


4) Dividir en datos de entrenamiento y prueba, para ello divide el texto en secuencias donde 
- la secuencia de la posición 0 a [seq_length] se considera de entrada, y 
- la secuencia de la posición  [seq_length+1] al final es la de salida

In [6]:
# determinar el largo máximo de la secuencia
if ((len(text)//101)<1000):
  seq_length = 50
else:
  seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print("Largo de secuencias: ", seq_length)
print("Ejemplos por época: ", examples_per_epoch)

# genera un vector de caracteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

#for i in char_dataset.take(5):
#  print(idx2char[i.numpy()])

# procesa para generar las secuencias el largo deseado
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# muestra ejemplo
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"b'\\xef\\xbb\\xbfa caballo regalado no se le mira el c"
'olmillo\\r\\na caballo regalado no se le miran los di'
'entes\\r\\na cada chancho le llega su san marti\\r\\na '
'confesion de parte relevo de prueba\\r\\nalli donde f'
'ueres haz lo que vieres\\r\\ndondequiera que fueres h'


In [7]:
# genera las secuencias de entrada y salida
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

datasetSeq = sequences.map(split_input_target)

print("DatasetSeq: ", datasetSeq, "\n")

# muestra ejemplo
for input_example, target_example in  datasetSeq.take(2):
  print ('Texto de Entrada: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Texto  de Salida:', repr(''.join(idx2char[target_example.numpy()])))

DatasetSeq:  <MapDataset shapes: ((50,), (50,)), types: (tf.int64, tf.int64)> 

Texto de Entrada:  "b'\\xef\\xbb\\xbfa caballo regalado no se le mira el "
Texto  de Salida: "'\\xef\\xbb\\xbfa caballo regalado no se le mira el c"
Texto de Entrada:  'olmillo\\r\\na caballo regalado no se le miran los d'
Texto  de Salida: 'lmillo\\r\\na caballo regalado no se le miran los di'


In [8]:
# muestra entrada y salida por cada caracter
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  Entrada: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  Salida Esperada: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  Entrada: 28 ('o')
  Salida Esperada: 25 ('l')
Step    1
  Entrada: 25 ('l')
  Salida Esperada: 26 ('m')
Step    2
  Entrada: 26 ('m')
  Salida Esperada: 22 ('i')
Step    3
  Entrada: 22 ('i')
  Salida Esperada: 25 ('l')
Step    4
  Entrada: 25 ('l')
  Salida Esperada: 25 ('l')


In [9]:
# genera 'batch' de secuencias que se van a procesar en el entrenamiento

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 100000

dataset = datasetSeq.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print("Dataset: ", dataset, "\n")


Dataset:  <BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)> 



5) Especificar el modelo de la RNN:


*   Modelo RNN básico:


In [0]:
# define el modelo RNN
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.SimpleRNN(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

print("Modelo RNN definido")


*   Modelo LSTM:


In [0]:
# define el modelo LSTM
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

print("Modelo LSTM definido")


*   Modelo GRU:


In [10]:
# define el modelo RNN
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

print("Modelo GRU definido")

Modelo GRU definido


Luego de seleccionado el modelo a armar, lo genera:

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# genera el modelo
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

# prepara variables auxiliares para el entrenamiento  de la RNN
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

# compila el modelo para el entrenamiento  de la RNN
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Forma vector predicción: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

print("\nModelo generado", model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           10240     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 40)            41000     
Total params: 3,989,544
Trainable params: 3,989,544
Non-trainable params: 0
_________________________________________________________________
(64, 50, 40) # (batch_size, sequence_length, vocab_size)
Forma vector predicción:  (64, 50, 40)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.6881292

Modelo generado <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f21b05b1160>


6) Entrenar la RNN:

In [12]:
# define donde se a almacenar la información de checkpoints

# Directory where the checkpoints will be saved
checkpoint_dir = './checkpoints/RNN_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

print("Checkpoints grabados en ", checkpoint_dir)

Checkpoints grabados en  ./checkpoints/RNN_training_checkpoints


In [13]:
# ejecutar el entrenamiento (poner antes en GPU)
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
18/18 [==============================] - 1s 42ms/step - loss: 3.5716
Epoch 2/100
18/18 [==============================] - 1s 38ms/step - loss: 2.7876
Epoch 3/100
18/18 [==============================] - 1s 38ms/step - loss: 2.3411
Epoch 4/100
18/18 [==============================] - 1s 42ms/step - loss: 2.0999
Epoch 5/100
18/18 [==============================] - 1s 38ms/step - loss: 1.9962
Epoch 6/100
18/18 [==============================] - 1s 38ms/step - loss: 1.9365
Epoch 7/100
18/18 [==============================] - 1s 39ms/step - loss: 1.8977
Epoch 8/100
18/18 [==============================] - 1s 38ms/step - loss: 1.8634
Epoch 9/100
18/18 [==============================] - 1s 39ms/step - loss: 1.8315
Epoch 10/100
18/18 [==============================] - 1s 40ms/step - loss: 1.8025
Epoch 11/100
18/18 [==============================] - 1s 39ms/step - loss: 1.7733
Epoch 12/100
18/18 [==============================] - 1s 38ms/step - loss: 1.7452
Epoch 13/100
18/18 [=====

7) Probar la RNN entrenada:

In [14]:
# recupera la información del último checkpoint
tf.train.latest_checkpoint(checkpoint_dir)

modelPred = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
modelPred.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
modelPred.build(tf.TensorShape([1, None]))

modelPred.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            10240     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 40)             41000     
Total params: 3,989,544
Trainable params: 3,989,544
Non-trainable params: 0
_________________________________________________________________


In [15]:
# define función auxiliar para devolver predicción de texto

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.3 # 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

# ejecuta el modeo usando como entrada un texto  (para ejemplo de MASTROPIERO)
print("\n\n--------------------------------------------------------------------\n\n")
print(generate_text(modelPred, start_string=u"a"))
print("\n\n--------------------------------------------------------------------\n\n")




--------------------------------------------------------------------


a sucia se lava en casa\r\nla sonrisa abre puertas\r\nes dificil que el tiempo no amanse\r\nno hay mal que dure cien a\xc3\xb1os\r\nno hay mal que cien a\xc3\xb1os dure\r\nno hay mal que dure cien a\xc3\xb1os\r\nno hay mal que cien a\xc3\xb1os dure\r\nno hay mal que dure cien a\xc3\xb1os\r\nno hay mal que cien a\xc3\xb1os dure\r\nno hay mal que dure cien a\xc3\xb1os dure ni bien que a ellos ature\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse\r\nmas vale trote que dure que galope que canse

In [17]:
# ejecuta el modeo usando como entrada un texto  (para ejemplo de MASTROPIERO)
print("\n\n--------------------------------------------------------------------\n\n")
print(generate_text(modelPred, start_string=u"no"))
print("\n\n--------------------------------------------------------------------\n\n")




--------------------------------------------------------------------


no se va lontano\r\npiensa mal y estaras acertado\r\nes camo para el otro como sandia en un carro\r\nde un cojo\r\ncae mas pronto el embustero que el agujero del mate\r\nmas viejo que andar a pie\r\nmas viejo que el agujero del mate\r\nmas viejo que andar a pie\r\nes un pecho fri\r\nes una iglesia abandonada no saca otro\r\ncon un clavo se saca otro\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun clavo saca a otro clavo\r\nun